### SPI 

#### data split 

In [1]:
import pandas as pd
path = '../data/spi10_2nrs.csv'
df = pd.read_csv(path)

In [2]:
from dataset import nrs2class 

In [3]:
df.pacu_nrs = df.pacu_nrs.map(lambda x: nrs2class(x,pain_type=2))

In [4]:
from sklearn.model_selection import train_test_split
xtr, xte, ytr, yte = train_test_split(df.iloc[:,:-1], df.iloc[:,-1], test_size=0.4, shuffle=True, stratify=df.iloc[:,-1], random_state=34)

In [5]:
trdf = xtr.join(ytr)
tedf = xte.join(yte)

In [6]:
trdf.to_csv('../data/spi10_nrs_train.csv',index=False)
tedf.to_csv('../data/spi10_nrs_test.csv',index=False)

#### Model shooting 

In [7]:
import torch
from dataset import Spi10_2nrs

trdt  = Spi10_2nrs('../data/spi10_nrs_train.csv')
tedt  = Spi10_2nrs('../data/spi10_nrs_test.csv')

trdl  = torch.utils.data.DataLoader(trdt, batch_size=8)
tedl  = torch.utils.data.DataLoader(tedt, batch_size=8)

In [4]:
# test
trit = iter(trdl)

x,y = next(trit)

In [15]:
from torch import nn
in_node = 5
lin_0  = nn.Linear(in_node,in_node*2)
gelu_0 = nn.GELU()
bn_0 = nn.BatchNorm1d(5)

NameError: name 'nn' is not defined

In [ ]:
from torch import nn
import torchaudio
class SimpleSPI10(torch.nn.Module):
    def __init__(self,in_node=5):
        super(SimpleSPI10,self).__init__()

        # feature_extraction
        bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
        self.wave2vec = bundle.get_model()
    
        encoder_layer = nn.TransformerEncoderLayer(d_model=32, nhead=4)
        tf_enc = nn.TransformerEncoder(encoder_layer, num_layers=3)


        conv1d_0 = nn.Conv1d(281,140,3)
        gelu_0 = nn.GELU()
        bn_0 = nn.BatchNorm1d(140)
        convb_0 = nn.Sequential(conv1d_0,gelu_0,bn_0)

        conv1d_1 = nn.Conv1d(140,70,3)
        gelu_1 = nn.GELU()
        bn_1 = nn.BatchNorm1d(70)
        convb_1 = nn.Sequential(conv1d_1,gelu_1,bn_1)

        conv1d_2 = nn.Conv1d(70,35,3)
        gelu_2 = nn.GELU()
        bn_2 = nn.BatchNorm1d(35)
        convb_2 = nn.Sequential(conv1d_2,gelu_2,bn_2)

        lin_3 = nn.Linear(26,14)
        relu_3 = nn.ReLU()
        linb_0 = nn.Sequential(lin_3,relu_3)

        lin_4 = nn.Linear(14,7)
        relu_4 = nn.ReLU()
        bn_4 = nn.BatchNorm1d(35)
        linb_1 = nn.Sequential(lin_4,relu_4,bn_4)

        fl_5 = nn.Flatten()
        lin_5 = nn.Linear(245,2)

        final = nn.Sequential(fl_5,lin_5)
        convbs = nn.Sequential(convb_0,convb_1,convb_2)
        lins   = nn.Sequential(linb_0,linb_1)

        self.blocks = nn.ModuleDict({'attention':tf_enc,'conv_blocks':convbs,'lin_blocks':lins,'fc':final})
        
    def forward(self,out):
        out, _ = self.wave2vec(out)
        out    = self.blocks['attention'](out)
        out    = self.blocks['conv_blocks'](out)
        out    = self.blocks['lin_blocks'](out)
        out    = self.blocks['fc'](out)
        return out

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
con0 = cons().to(device)
loss = nn.CrossEntropyLoss()
opt  = torch.optim.Adam(con0.parameters() )

In [ ]:
def train(dl,model,lossf,opt):
    for x,y in dl:
        x,y = x.to(device),y.to(device)
        pre = model(x)
        loss = lossf(pre,y)

        opt.zero_grad()
        loss.backward()
        opt.step()

def test(dl,model,lossf):
    model.eval()
    size, acc , losses = len(dl.dataset) ,0,0
    with torch.no_grad():
        for x,y in dl:
            x,y = x.to(device),y.to(device)
            pre = model(x)
            loss = lossf(pre,y)

            acc += (pre.argmax(1)==y).type(torch.float).sum().item()
            losses += loss.item()
    print(f'{acc/size} : {losses/size}')

from tqdm import tqdm
for _ in range(120):
    train(trdl,con0,loss,opt)
    test(tedl,con0,loss)